In [33]:
import pandas as pd
from sqlalchemy import create_engine
from getpass import getpass
from urllib.parse import quote_plus

pd.set_option('display.max_columns', None)

In [34]:
host = 'aws-1-us-east-1.pooler.supabase.com'.strip()
port = '5432'.strip()
database = 'postgres'.strip()
user = 'candidato.jdazostyahhxukbmxybw'.strip()
password = 'DesafioInov@2026!'
password_cod = quote_plus(password)

connection_string = f"postgresql+psycopg2://{user}:{password_cod}@{host}:{port}/{database}"

try:
    engine = create_engine(connection_string)
    with engine.connect() as connection:
        print("Conexão bem-sucedida!")
except Exception as e:
    print(f"Erro ao conectar: {e}")

Conexão bem-sucedida!


In [31]:
'''
Questão 1: Existem pagamentos e liquidações sem um empenho associado?
'''

query_pag1 = """
SELECT * FROM pagamento WHERE pagamento.id_empenho IS NULL
"""

query_liq1 = """
SELECT * FROM liquidacao_nota_fiscal WHERE liquidacao_nota_fiscal.id_empenho IS NULL
"""

resultpag1 = pd.read_sql(query_pag1, engine)
resultliq1 = pd.read_sql(query_liq1, engine)
                        
print(f"Quantidade de pagamentos sem empenhos é de {len(resultpag1)}");
display(resultpag1)

print(f"Quantidade de liquidações sem empenhos é de {len(resultliq1)}");
display(resultliq1)



Quantidade de pagamentos sem empenhos é de 0


,id_empenho,id_pagamento,datapagamentoempenho,valor


Quantidade de liquidações sem empenhos é de 0


,id_liquidacao_empenhonotafiscal,chave_danfe,data_emissao,valor,id_empenho


In [36]:
'''
Questão 2: Há algum empenho que não esteja associado a nenhum contrato? 
'''

query_emp2 = """
SELECT * FROM empenho WHERE empenho.id_contrato IS NULL
"""
result = pd.read_sql(query_emp2, engine)
print(f"Número de empenhos que não possuem contratos associados: {len(result)}")
display(result)

Número de empenhos que não possuem contratos associados: 0


,id_empenho,ano,data_empenho,cpfcnpjcredor,credor,valor,id_entidade,id_contrato


In [37]:
'''
Questão 3: Há algum contrato que não possui entidade ou fornecedor?
'''

query_ent3 = """
SELECT * FROM contrato WHERE contrato.id_entidade IS NULL
"""

query_for3 = """
SELECT * FROM contrato WHERE contrato.id_fornecedor IS NULL
"""

resultent3 = pd.read_sql(query_ent3, engine)
resultfor3 = pd.read_sql(query_for3, engine)

print(f"Número de contratos que não possuem entidades associadas:{len(resultent3)}")
display(resultent3)

print(f"Número de contratos que não possuem fornecedores associados: {len(resultfor3)}")
display(resultfor3)

Número de contratos que não possuem entidades associadas:0


,id_contrato,valor,data,objeto,id_entidade,id_fornecedor


Número de contratos que não possuem fornecedores associados: 0


,id_contrato,valor,data,objeto,id_entidade,id_fornecedor


In [41]:
'''
Questão 4: Há algum registro no qual o valor do empenho seja superior que o valor do contrato?
'''

query_reg4 = """
SELECT 
    e.id_empenho, 
    e.credor, 
    e.valor as valor_empenho,

    c.id_contrato,
    c.valor as valor_contrato

FROM contrato c INNER JOIN empenho e ON c.id_contrato = e.id_contrato
"""

result = pd.read_sql(query_reg4, engine)

empenho_superior = result[result['valor_empenho']>result['valor_contrato']]
empenho_superior['diferenca'] = empenho_superior['valor_empenho'] - result['valor_contrato']

print(f"O número de empenhos com valores maiores do que o do contrato é de: {len(empenho_superior)}")
display(empenho_superior.sort_values('diferenca', ascending=False))

O número de empenhos com valores maiores do que o do contrato é de: 150


,id_empenho,credor,valor_empenho,id_contrato,valor_contrato,diferenca
359,EMP-11,Saude Vida Distribuidora,82476.26,1013,49985.61,32490.65
480,EMP-224,TechServer Solucoes,82166.98,1139,49798.17,32368.81
388,EMP-140,Transportes Rapido,81629.94,1046,49472.69,32157.25
371,EMP-122,LimpMax Servicos,80642.35,1026,48874.15,31768.20
487,EMP-231,Merenda Boa Alimentos,80564.57,1147,48827.01,31737.56
...,...,...,...,...,...,...
463,EMP-215,Eventos & Promo,3948.37,1129,2392.95,1555.42
445,EMP-199,ConstruRio Engenharia,3266.16,1110,1979.49,1286.67
481,EMP-225,ConstruRio Engenharia,2519.62,1140,1527.04,992.58
442,EMP-196,Saude Vida Distribuidora,2502.92,1107,1516.92,986.00


In [7]:
'''
Questão 5: Existem contratos sem entidade ou fornecedor?
'''

query_contratos = """
SELECT * FROM contrato
WHERE id_contrato IS NULL
"""

query_forn = """
SELECT * FROM contrato
WHERE id_fornecedor IS NULL
"""

df_resultado = pd.read_sql(query_contratos, engine)
df_resultado2 = pd.read_sql(query_forn, engine)

print("Quantidade de contratos sem entidade:")
display(df_resultado)
print("Quantidade de contratos sem fornecedores:")
display(df_resultado2)

Quantidade de contratos sem entidade:


,id_contrato,valor,data,objeto,id_entidade,id_fornecedor


Quantidade de contratos sem fornecedores:


,id_contrato,valor,data,objeto,id_entidade,id_fornecedor


In [12]:
'''
Questão 6: O valor de algum contrato é maior que o valor empenhado para a despesa pública?
'''

query_contrato = """
SELECT id_contrato, valor as valor_teto_contrato FROM contrato
"""

query_empenho = """
SELECT id_empenho, id_contrato, valor as valor_reservado_empenho FROM empenho
"""

df_contratos = pd.read_sql(query_contrato, engine)
df_empenhos = pd.read_sql(query_empenho, engine)

analise_teto = pd.merge(df_contratos, df_empenhos, on = 'id_contrato', how='inner')

contratos = analise_teto[analise_teto['valor_reservado_empenho'] > analise_teto['valor_teto_contrato']]

print(f"Empenhos que estouraram o valor do contrato: {len(contratos)}")
display(contratos)


Empenhos que estouraram o valor do contrato: 150


,id_contrato,valor_teto_contrato,id_empenho,valor_reservado_empenho
0,1001,20980.58,EMP-1,34617.96
1,1002,26021.35,EMP-10,42935.23
2,1003,25384.45,EMP-100,41884.34
3,1004,15924.19,EMP-101,26274.91
4,1005,30930.82,EMP-102,51035.85
...,...,...,...,...
145,1146,8662.18,EMP-230,14292.60
146,1147,48827.01,EMP-231,80564.57
147,1148,37448.38,EMP-232,61789.83
148,1149,7040.31,EMP-233,11616.51


In [18]:
'''
Questão 7: Há algum valor pago que é maior que o empenho?
'''

query = """
SELECT 
    p.id_pagamento,
    p.datapagamentoempenho AS data_pagamento, 
    p.valor AS valor_pago,
    
    e.id_empenho,
    e.valor AS valor_empenhado,
    
    c.id_contrato,
    c.valor AS valor_teto_contrato

FROM pagamento p
INNER JOIN empenho e ON p.id_empenho = e.id_empenho
INNER JOIN contrato c ON e.id_contrato = c.id_contrato
"""



df_raio_x = pd.read_sql(query, engine)
pagamentos_estourados = df_raio_x [df_raio_x['valor_pago'] > df_raio_x['valor_empenhado']]

pagamentos_estourados['diferenca'] = pagamentos_estourados['valor_pago'] - pagamentos_estourados['valor_empenhado']

print(f"Quantidade de pagamentos acima do teto do empenho: {len(pagamentos_estourados)}")
display(pagamentos_estourados.sort_values('diferenca', ascending=False))


Quantidade de pagamentos acima do teto do empenho: 225


,id_pagamento,data_pagamento,valor_pago,id_empenho,valor_empenhado,id_contrato,valor_teto_contrato,diferenca
426,PGT-430,2024-01-19,45603.04,EMP-430,1024.70,1367,1024.70,44578.34
274,PGT-277,2024-02-04,46520.70,EMP-277,2261.26,1197,2261.26,44259.44
283,PGT-286,2024-01-14,49633.66,EMP-286,6451.56,1207,6451.56,43182.10
488,PGT-492,2024-01-21,49394.11,EMP-492,6657.12,1435,6657.12,42736.99
422,PGT-426,2024-02-03,49232.11,EMP-426,6557.90,1362,6557.90,42674.21
...,...,...,...,...,...,...,...,...
90,PGT-93,2024-02-05,25764.12,EMP-93,25340.64,1491,25340.64,423.48
17,PGT-19,2023-12-31,17427.94,EMP-19,17109.77,1100,10369.56,318.17
237,PGT-240,2024-01-24,49220.70,EMP-240,49051.89,1157,49051.89,168.81
165,PGT-168,2024-01-27,8928.78,EMP-168,8884.69,1076,5384.66,44.09
